In [1]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx

import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re
from matplotlib import rcParams
from bs4 import BeautifulSoup
import time

In [2]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index

            index += 1

    return -1

In [3]:
def get_poll_xml1_draft(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com/draft/NHL_%i_entry.html" % int(poll_id)
    
    return requests.get(url).text

In [4]:
#http://www.hockey-reference.com/players/c/crosbsi01.html
def get_poll_xml1_player(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com" + poll_id

    return requests.get(url).text

In [5]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False


In [ ]:
for dft_year in range(1989,2016):
    csv_file='dfplayer'+str(dft_year)+'.csv'
    dataplayer=pd.read_csv(csv_file,index_col=False)
    print 'dft_year'
    print dft_year
    start = time.time()
    
    df_frames=[]
    for ith_player in range(0,len(dataplayer['url'])):
        player_url=dataplayer['url'][ith_player][:-5]
        player_name=dataplayer['name'][ith_player]
        player_height=dataplayer['height'][ith_player]
        player_weight=dataplayer['weight'][ith_player]
        player_shot=dataplayer['shot'][ith_player]
        player_pos=dataplayer['pos'][ith_player]

        try:
            player_dft=int(re.sub("\D", "", dataplayer['draft'][ith_player]))
        except:
            player_dft=dataplayer['draft'][ith_player]
        print ith_player
        print player_name
        print 
        for year in range(dft_year+2,2018):
            time.sleep(0.1)
            player_data=[]
            url='http://www.hockey-reference.com'+player_url+'/gamelog/'+str(year)+'/'
            page=requests.get(url).text
            pagesoap=BeautifulSoup(page, "lxml")
            strsoap=str(pagesoap)
            posseStr=(find_str(strsoap, "Defensive Zone Faceoffs / (Offensive Zone Faceoffs + Defensive Zone Faceoffs),<br />that took place while on the ice"))

            posStr=(find_str(strsoap, "Awards</th>\n</tr>\n</thead>\n<tbody>"))
            posEnd=(find_str(strsoap, "</tr>\n</tfoot>\n</table>\n</div>\n</div>\n</div><!-- div.table_wrapper -->\n<!-- sr_mobile_1 -->"))
            posseEnd=(find_str(strsoap, " <h2>NHL Miscellaneous</h2>"))

            strlimitPos=strsoap[(posseStr+116):posseEnd]

            win_lose=['gm_result']
            for m in re.finditer('data-stat="game_result"', strlimitPos):
                if strlimitPos[(m.end()+3):(m.end()+5)]=='</':
                    break
                elif strlimitPos[(m.end())+1:(m.end()+2)]=='L' or strlimitPos[(m.end())+1:(m.end()+2)]=='W':
                    win_lose.append(strlimitPos[(m.end())+1:(m.end()+2)])

            player_data.append(win_lose)

            ages=['age']
            for m in re.finditer('data-stat="age"', strlimitPos):
                if strlimitPos[(m.end()+3):(m.end()+5)]=='</':
                    break
                elif strlimitPos[(m.end()+1):(m.end()+3)].isdigit() and strlimitPos[(m.end()+4):(m.end()+7)].isdigit():
                    age_data=float(strlimitPos[(m.end()+4):(m.end()+7)])/365.0
                    ages.append(round(float(strlimitPos[(m.end()+1):(m.end()+3)])+age_data,2))

            player_data.append(ages)

            num_games=['ith_game']
            for m in re.finditer('data-stat="game_season"', strlimitPos):    
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    num_games.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    num_games.append(strlimitPos[(m.end()+1):(m.end()+2)])

            player_data.append(num_games)

            teams_for=['team_for']
            for m in re.finditer('data-stat="team_id"', strlimitPos):
                if strlimitPos[(m.end()+17):(m.end()+20)].isupper():
                    teams_for.append(strlimitPos[(m.end()+17):(m.end()+20)])

            player_data.append(teams_for)

            teams_agt=['team_agt']
            for m in re.finditer('data-stat="opp_id"', strlimitPos):

                if strlimitPos[(m.end()+17):(m.end()+20)].isupper():
                    teams_agt.append(strlimitPos[(m.end()+17):(m.end()+20)])

            player_data.append(teams_agt)

            dates_game=['date_game']
            for m in re.finditer('data-stat="date_game"', strlimitPos):
                if strlimitPos[(m.end()+40):(m.end()+44)].isdigit() and strlimitPos[(m.end()+45):(m.end()+47)].isdigit() and \
                    strlimitPos[(m.end()+48):(m.end()+50)].isdigit():
                    dates_game.append(strlimitPos[(m.end()+40):(m.end()+50)])

            player_data.append(dates_game)

            loc_game=['gm_loc']
            for m in re.finditer('data-stat="game_location"', strlimitPos):
                if strlimitPos[(m.end()+3):(m.end()+5)]=='td' or strlimitPos[(m.end()+4):(m.end()+6)]=='td':
                    if strlimitPos[(m.end()+1)]=='@':
                        loc_game.append('Away')
                    elif strlimitPos[(m.end()+1)]=='<':
                        loc_game.append('Home')

            player_data.append(loc_game)

            goals_all=['goals_all']
            for m in re.finditer('data-stat="goals"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    goals_all.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(goals_all)

            assists_all=['assists_all']
            for m in re.finditer('data-stat="assists"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    assists_all.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(assists_all)


            plus_minus=['plus_minus']
            for m in re.finditer('data-stat="plus_minus"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    plus_minus.append(strlimitPos[(m.end()+1):(m.end()+2)])
                elif strlimitPos[(m.end()+2):(m.end()+3)].isdigit():
                    plus_minus.append('-'+strlimitPos[(m.end()+2):(m.end()+3)])
            player_data.append(plus_minus)

            pen_min=['pen_min']
            for m in re.finditer('data-stat="pen_min"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    pen_min.append(strlimitPos[(m.end()+1):(m.end()+3)])

                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    pen_min.append(strlimitPos[(m.end()+1):(m.end()+2)])

            player_data.append(pen_min)

            goals_ev=['goals_ev']
            for m in re.finditer('data-stat="goals_ev"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    goals_ev.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(goals_ev)

            goals_pp=['goals_pp']
            for m in re.finditer('data-stat="goals_pp"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    goals_pp.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(goals_pp)

            goals_sh=['goals_sh']
            for m in re.finditer('data-stat="goals_sh"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    goals_sh.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(goals_sh)

            assists_pp=['assists_pp']
            for m in re.finditer('data-stat="assists_pp"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    assists_pp.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(assists_pp)

            assists_ev=['assists_ev']
            for m in re.finditer('data-stat="assists_ev"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    assists_ev.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(assists_ev)

            assists_sh=['assists_sh']
            for m in re.finditer('data-stat="assists_sh"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    assists_sh.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(assists_sh)

            num_shifts=['num_shifts']
            for m in re.finditer('data-stat="shifts"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    num_shifts.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    num_shifts.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(num_shifts)

            time_on_ice=['time_on_ice']
            for m in re.finditer('data-stat="time_on_ice"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit() and strlimitPos[(m.end()+4):(m.end()+6)].isdigit():
                    sec=float(strlimitPos[(m.end()+4):(m.end()+6)])/60.0
                    time_on_ice.append(round(float(strlimitPos[(m.end()+1):(m.end()+3)])+sec,2))
                if strlimitPos[(m.end()+1):(m.end()+2)].isdigit() and strlimitPos[(m.end()+3):(m.end()+5)].isdigit():
                    sec=float(strlimitPos[(m.end()+3):(m.end()+5)])/60.0
                    time_on_ice.append(round(float(strlimitPos[(m.end()+1):(m.end()+2)])+sec,2))
            player_data.append(time_on_ice)

            hits_all=['hits_all']
            for m in re.finditer('data-stat="hits_all"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    hits_all.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    hits_all.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(hits_all)

            blocks_all=['blocks_all']
            for m in re.finditer('data-stat="blocks_all"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    blocks_all.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    blocks_all.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(blocks_all)

            fc_win=['fc_win']
            for m in re.finditer('data-stat="faceoff_wins_all"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    fc_win.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    fc_win.append(strlimitPos[(m.end()+1):(m.end()+2)])
            player_data.append(fc_win)

            fc_loss=['fc_loss']
            for m in re.finditer('data-stat="faceoff_losses_all"', strlimitPos):
                if strlimitPos[(m.end()+1):(m.end()+3)].isdigit():
                    fc_loss.append(strlimitPos[(m.end()+1):(m.end()+3)])
                elif strlimitPos[(m.end()+1):(m.end()+2)].isdigit():
                    fc_loss.append(strlimitPos[(m.end()+1):(m.end()+2)])        
            player_data.append(fc_loss)

            # year
            yr=[year]*len(win_lose)
            yr.insert(0,'Year')
            player_data.append(yr)

            # name
            names=[player_name]*len(win_lose)
            names.insert(0,'Name')
            player_data.append(names)

            # player_url
            urls=[player_url[9:]]*len(win_lose)
            urls.insert(0,'Url')
            player_data.append(urls)


            # player_height
            heights=[player_height]*len(win_lose)
            heights.insert(0,'Height')
            player_data.append(heights)


            # player_weight
            weights=[player_weight]*len(win_lose)
            weights.insert(0,'Weight')
            player_data.append(weights)

            # player_shot
            player_shots=[player_shot]*len(win_lose)
            player_shots.insert(0,'ShotHand')
            player_data.append(player_shots)

            # player_pos
            player_posses=[player_pos]*len(win_lose)
            player_posses.insert(0,'Position')
            player_data.append(player_posses)

            # player_drafts
            player_drafts=[player_dft]*len(win_lose)
            player_drafts.insert(0,'Drafts')
            player_data.append(player_drafts)

            # player_drafts
            dft_years=[dft_year]*len(win_lose)
            dft_years.insert(0,'draft_years')
            player_data.append(dft_years)



            player_data_T=map(None,*player_data)
            df=pd.DataFrame(player_data_T[1:],columns=player_data_T[0])
            df = df[df['ith_game'] > 0]
            df_frames.append(df)


    new_df=pd.concat(df_frames)
    new_df.to_csv('player_games_data_'+str(dft_year)+'.csv')
    end = time.time()
    print(end - start)

dft_year
1992
0
Roman Hamrlik

1
Alexei Yashin

